# Berend Botje Skills powered by ChatGPT en eigen data

De notebook is geinspireerd op het werk uit de OpenAI Cookbook.

De werkwijze is als volgt:
- **Setup:** 
    - Initieren van de variabelen en het laden van de data bestanden
- **Bouwen van het fundament:**
    - Setup van een vector database die vectoren en data accepteert
    - Laden van de dataset, chunken van de data zodat embedding mogelijk is en de data in de vectorendatabase kan worden opgeslagen. 
- **Op weg naar het product:**
    - Toevoegen van een retrieval stap waar gebruikers vragen stellen en we de meest relevante entries teruggeven 
    - Samenvatten van de zoekresultaten met GPT-3
    - Testen van de basis Q&A app in Streamlit
- **Bouwen van Berend:**
    - Maken van een Assistant class om context te kunnen beheren en we kunnen interacteren met Berend de Bot
    - Berend Bot gebruiken om de vragen te beantwoorden door gebruik te maken van de semantische zoek context.
    - Testen van de basis BerenBot app in Streamlit
    

In [1]:
%load_ext autoreload
%autoreload 2

## Setup

Setup van de bibliotheken en omgevingsvariabelen


In [2]:
import openai
import os
import requests
import numpy as np
import pandas as pd
from typing import Iterator
import tiktoken
import textract
from numpy import array, average

from database import get_redis_connection

# Set our default models and chunking size
from config import COMPLETIONS_MODEL, EMBEDDINGS_MODEL, CHAT_MODEL, TEXT_EMBEDDING_CHUNK_SIZE, VECTOR_FIELD_NAME

# Ignore unclosed SSL socket warnings - optional in case you get these errors
import warnings

warnings.filterwarnings(action="ignore", message="unclosed", category=ImportWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [3]:
pd.set_option('display.max_colwidth', 0)

In [4]:
data_dir = os.path.join(os.curdir,'data')
pdf_files = sorted([x for x in os.listdir(data_dir) if 'DS_Store' not in x])
pdf_files

['230222 Verslag Stuurgroep Fieldlab TV.pdf',
 'Audiobestand.pdf',
 'Docentenhandleiding Burgerschap).pdf']

## Het bouwen van het fundament

### Opslag

We're going to use Redis as our database for both document contents and the vector embeddings. You will need the full Redis Stack to enable use of Redisearch, which is the module that allows semantic search - more detail is in the [docs for Redis Stack](https://redis.io/docs/stack/get-started/install/docker/).

To set this up locally, you will need to install Docker and then run the following command: ```docker run -d --name redis-stack -p 6379:6379 -p 8001:8001 redis/redis-stack:latest```.

The code used here draws heavily on [this repo](https://github.com/RedisAI/vecsim-demo).

After setting up the Docker instance of Redis Stack, you can follow the below instructions to initiate a Redis connection and create a Hierarchical Navigable Small World (HNSW) index for semantic search.

In [5]:
# Setup Redis
from redis import Redis
from redis.commands.search.query import Query
from redis.commands.search.field import (
    TextField,
    VectorField,
    NumericField
)
from redis.commands.search.indexDefinition import (
    IndexDefinition,
    IndexType
)

redis_client = get_redis_connection()

In [6]:
# Constants
VECTOR_DIM = 1536 #len(data['title_vector'][0]) # length of the vectors
#VECTOR_NUMBER = len(data)                 # initial number of vectors
PREFIX = "sportsdoc"                            # prefix for the document keys
DISTANCE_METRIC = "COSINE"                # distance metric for the vectors (ex. COSINE, IP, L2)

In [7]:
# Create search index

# Index
INDEX_NAME = "f1-index"           # name of the search index
VECTOR_FIELD_NAME = 'content_vector'

# Define RediSearch fields for each of the columns in the dataset
# This is where you should add any additional metadata you want to capture
filename = TextField("filename")
text_chunk = TextField("text_chunk")
file_chunk_index = NumericField("file_chunk_index")

# define RediSearch vector fields to use HNSW index

text_embedding = VectorField(VECTOR_FIELD_NAME,
    "HNSW", {
        "TYPE": "FLOAT32",
        "DIM": VECTOR_DIM,
        "DISTANCE_METRIC": DISTANCE_METRIC
    }
)
# Add all our field objects to a list to be created as an index
fields = [filename,text_chunk,file_chunk_index,text_embedding]

In [8]:
redis_client.ping()

True

In [9]:
# Optional step to drop the index if it already exists
#redis_client.ft(INDEX_NAME).dropindex()

# Check if index exists
try:
    redis_client.ft(INDEX_NAME).info()
    print("Index already exists")
except Exception as e:
    print(e)
    # Create RediSearch Index
    print('Not there yet. Creating')
    redis_client.ft(INDEX_NAME).create_index(
        fields = fields,
        definition = IndexDefinition(prefix=[PREFIX], index_type=IndexType.HASH)
    )

Index already exists


### Laden van de data

We'll load up our PDFs and do the following
- Initiate our tokenizer
- Run a processing pipeline to:
    - Mine the text from each PDF
    - Split them into chunks and embed them
    - Store them in Redis

In [10]:
# The transformers.py file contains all of the transforming functions, including ones to chunk, embed and load data
# For more details the file and work through each function individually
from transformers import handle_file_string

In [11]:
%%time
# Initialise tokenizer
tokenizer = tiktoken.get_encoding("cl100k_base")
print(pdf_files)

print(data_dir)
# data_dir = "data/"

for pdf_file in pdf_files:
    pdf_path = os.path.join(data_dir,pdf_file)
    print(pdf_path)
    text = textract.process(pdf_path)
    # Extract the raw text from each PDF using textract
    # text = textract.process(pdf_path, method='pdfminer')
    handle_file_string((pdf_file,text.decode("utf-8")),tokenizer,redis_client,VECTOR_FIELD_NAME,INDEX_NAME)
    
    
    

['230222 Verslag Stuurgroep Fieldlab TV.pdf', 'Audiobestand.pdf', 'Docentenhandleiding Burgerschap).pdf']
.\data
.\data\230222 Verslag Stuurgroep Fieldlab TV.pdf
.\data\Audiobestand.pdf
.\data\Docentenhandleiding Burgerschap).pdf
CPU times: total: 1.53 s
Wall time: 7.01 s


In [ ]:
%%time
# This step takes about 5 minutes

# Initialise tokenizer
tokenizer = tiktoken.get_encoding("cl100k_base")
print(pdf_files)

# data_dir = "data/"

# Process each PDF file and prepare for embedding
for pdf_file in pdf_files:
    
    pdf_path = os.path.join(data_dir,pdf_file)
    print(pdf_path)
    
    # Extract the raw text from each PDF using textract
    text = textract.process(pdf_path) #, method='pdfminer'
    # print(text)
    
    # Chunk each document, embed the contents and load to Redis
    handle_file_string((pdf_file,text.decode("utf-8")),tokenizer,redis_client,VECTOR_FIELD_NAME,INDEX_NAME)

In [12]:
# Check that our docs have been inserted
redis_client.ft(INDEX_NAME).info()['num_docs']

'745'

## Make it a product

Now we can test that our search works as intended by:
- Querying our data in Redis using semantic search and verifying results
- Adding a step to pass the results to GPT-3 for summarisation

In [13]:
from database import get_redis_results

In [14]:
%%time

f1_query="Welke Thema's zijn er te vinden in de docentenhandleiding Burgerschap? Geef met bullits de titels van deze Thema's"

result_df = get_redis_results(redis_client,f1_query,index_name=INDEX_NAME)
result_df.head(2)

CPU times: total: 31.2 ms
Wall time: 284 ms


,id,result,certainty
0,0,Filename is: Docentenhandleiding Burgerschap).pdf;,0.140407323837
1,1,"De docent uren volgt de student vanuit.\r Spreker 10\r Studenten zeker, ja?\r Spreker 13\r Je keuze, dat maakt het student daar ingaan. Heb je ook helemaal terug?\r Spreker 7\r Klopt, ja klopt.\r Spreker 11\r We hebben met.\r \r Spreker 9\r Marcel van de week over gesproken over.\r Spreker 5\r Mochten de toe.\r Spreker 9\r Nou het kiezen van een aantal focuspunten, bijvoorbeeld dat we daarin kunnen meelopen met de werkgroepen. Dat zou bijvoorbeeld ook wel heel mooi zijn, h�? Vind je dat het onderwijs en de werkgroepen een beetje gelijk op laat gaan en dan op gegeven moment die kruisbestuiving dan ook krijgt, want dan zit je inhoudelijk in ieder geval goed, want soms moet je het bedrijf ook gewoon een. Beetje opleiden op wat? Wat �berhaupt is er allemaal op dit moment aan kennis aanwezig en hoe kunnen we daar graag van?\r Spreker 2\r Vragen over dit punt. Ook gezicht inhoudelijk. Maakt het bedrijf kant. Dank u zust�ndig voor deze update. Dan gaan we door naar grego denk ik h�, voor de financi�le voortgang.\r Spreker 1\r Ja nee. Ja, dat klopt. Ja met name dit. Even kijken waar ik tot en met eergisteren gisteren. Nou.\r Spreker 7\r Dus dat patch 3 zit erin?\r Spreker 4\r Ja ja.\r Spreker 1\r En daar nou ja, zoals ik al In het begin met met.",0.182792901993


In [15]:
# Build a prompt to provide the original query, the result and ask to summarise for the user
summary_prompt = '''Maak een samenvatting die een bullitlijst als antwoord levert op de zoekvraag die de gebruiker heeft gestuurd.
Search query: SEARCH_QUERY_HERE
Search result: SEARCH_RESULT_HERE
Samenvatting:
'''
summary_prepped = summary_prompt.replace('SEARCH_QUERY_HERE',f1_query).replace('SEARCH_RESULT_HERE',result_df['result'][0])
summary = openai.Completion.create(engine=COMPLETIONS_MODEL,prompt=summary_prepped,max_tokens=500)
# Response provided by GPT-3
print(summary['choices'][0]['text'])


• Thema 1: Wat is burgerschap?
• Thema 2: Essentiële basisvaardigheden leren
• Thema 3: Hoe burgerschap een invloed heeft op het dagelijks leven
• Thema 4: De rol van burgerschap in onderwijs
• Thema 5: Maatschappelijke inclusie en diversiteit
• Thema 6: Ethische en sociale verantwoordelijkheden
• Thema 7: Confronteren van stereotypen


### Search

Now that we've got our knowledge embedded and stored in Redis, we can now create an internal search application. Its not sophisticated but it'll get the job done for us.

In the directory containing this app, execute ```streamlit run search.py```. This will open up a Streamlit app in your browser where you can ask questions of your embedded data.

__Example Questions__:
- what is the cost cap for a power unit in 2023
- what should competitors include on their application form

## Build your moat

The Q&A was useful, but fairly limited in the complexity of interaction we can have - if the user asks a sub-optimal question, there is no assistance from the system to prompt them for more info or conversation to lead them down the right path.

For the next step we'll make a Chatbot using the Chat Completions endpoint, which will:
- Be given instructions on how it should act and what the goals of its users are
- Be supplied some required information that it needs to collect
- Go back and forth with the customer until it has populated that information
- Say a trigger word that will kick off semantic search and summarisation of the response

For more details on our Chat Completions endpoint and how to interact with it, please check out the docs [here](https://platform.openai.com/docs/guides/chat).

### Framework

This section outlines a basic framework for working with the API and storing context of previous conversation "turns". Once this is established, we'll extend it to use our retrieval endpoint.

In [17]:
# A basic example of how to interact with our ChatCompletion endpoint
# It requires a list of "messages", consisting of a "role" (one of system, user or assistant) and "content"
question = 'Hoe kun je mij helpen?'


completion = openai.ChatCompletion.create(
  model="gpt-4",
  messages=[
    {"role": "user", "content": question}
  ]
)
print(f"{completion['choices'][0]['message']['role']}: {completion['choices'][0]['message']['content']}")

assistant: Als een kunstmatige intelligentie van OpenAI kan ik u ondersteunen door informatie te verstrekken, vragen te beantwoorden, u te helpen bij het organiseren van taken, advies te geven over verschillende onderwerpen en nog veel meer. I kan u helpen met zaken zoals tijdbeheer, productiviteit, algemene kennisvragen en nog veel meer. Let op: hoewel ik probeer zo correct en behulpzaam mogelijk te zijn, ben ik nog steeds een AI en kan ik menselijke fouten niet volledig uitsluiten.


In [18]:
from termcolor import colored

# A basic class to create a message as a dict for chat
class Message:
    
    
    def __init__(self,role,content):
        
        self.role = role
        self.content = content
        
    def message(self):
        
        return {"role": self.role,"content": self.content}
        
# Our assistant class we'll use to converse with the bot
class Assistant:
    
    def __init__(self):
        self.conversation_history = []

    def _get_assistant_response(self, prompt):
        
        try:
            completion = openai.ChatCompletion.create(
              model="gpt-4",  # gpt-3.5-turbo
              messages=prompt
            )
            
            response_message = Message(completion['choices'][0]['message']['role'],completion['choices'][0]['message']['content'])
            return response_message.message()
            
        except Exception as e:
            
            return f'Request failed with exception {e}'

    def ask_assistant(self, next_user_prompt, colorize_assistant_replies=True):
        [self.conversation_history.append(x) for x in next_user_prompt]
        assistant_response = self._get_assistant_response(self.conversation_history)
        self.conversation_history.append(assistant_response)
        return assistant_response
            
        
    def pretty_print_conversation_history(self, colorize_assistant_replies=True):
        for entry in self.conversation_history:
            if entry['role'] == 'system':
                pass
            else:
                prefix = entry['role']
                content = entry['content']
                output = colored(prefix +':\n' + content, 'green') if colorize_assistant_replies and entry['role'] == 'assistant' else prefix +':\n' + content
                print(output)

In [19]:
# Initiate our Assistant class
conversation = Assistant()

# Create a list to hold our messages and insert both a system message to guide behaviour and our first user question
messages = []
system_message = Message('system','Jij bent een behulpzame assistent die kan assisteren bij het maken van een lesplan, en bij het notuleren.')
user_message = Message('user','Wat kun je voor mij doen?')
messages.append(system_message.message())
messages.append(user_message.message())
messages

[{'role': 'system',
  'content': 'Jij bent een behulpzame assistent die kan assisteren bij het maken van een lesplan, en bij het notuleren.'},
 {'role': 'user', 'content': 'Wat kun je voor mij doen?'}]

In [20]:
# Get back a response from the Chatbot to our question
response_message = conversation.ask_assistant(messages)
print(response_message['content'])

Als assistent kan ik je op verschillende manieren helpen:

1. Lesplan Maken: Ik kan je helpen bij het organiseren en maken van een gedetailleerd lesplan. Dit kan variëren van het plannen van een enkele les tot het ontwerpen van een volledig curriculum, inclusief het indelen van activiteiten, het vaststellen van leerdoelen en het aangeven van eventuele benodigde materialen.

2. Notuleren: Ik kan je helpen bij het bijhouden van vergaderingen of klassendiscussies. Dit kan inhouden dat ik belangrijke punten, genomen beslissingen en toegewezen taken noteer, of samenvattingen maak na afloop.

3. Beheren van agenda's: Ik kan helpen bij het plannen en bijhouden van afspraken, belangrijke data en deadlines, ervoor zorgen dat je georganiseerd blijft.

4. Onlinediensten: Ik kan je helpen bij het navigeren door het digitale landschap, zoals het zoeken naar bronnen voor lesmateriaal, het gebruik van educatieve apps en tools, en het begrijpen van het gebruik van online platforms voor leren en onderw

In [21]:
next_question = 'Kun je voor mij op basis van de docentenhandleiding Burgerschap een lesplan samenstellen over Thema 1, les 1?'

# Initiate a fresh messages list and insert our next question
messages = []
user_message = Message('user',next_question)
messages.append(user_message.message())
response_message = conversation.ask_assistant(messages)
print(response_message['content'])

Natuurlijk, ik ben blij dat ik kan helpen bij het maken van je lesplan. Omdat ik een AI assistent ben, kan ik geen fysieke documenten of bestanden openen, maar ik kan je zeker helpen bij het opzetten van een algemeen format voor je lesplan. 

Hierbij kan je een idee krijgen van hoe het kan worden gestructureerd:

-----
**Lesplan Burgerschap**

**Thema 1, Les 1**

**Leerdoelen:**
- Pain punt 1 (afgeleid van de docentenhandleiding)
- Punt 2 ....
- Punt 3 ....

**Inleiding activiteit:** (5-10 min)
Een korte activiteit of dialoog om het onderwerp van de les in te leiden.

**Hoofdactiviteit:** (30-40 minuten)
- Instructie: Uitleg van het belangrijkste concept of vaardigheid voor deze les.
- Oefening: Praktische activiteit voor leerlingen om het concept of de vaardigheid dat is uitgelegd in de instructie te oefenen.

**Afsluitende activiteit:** (10-15 min)
Een activiteit of discussie om de behandelde onderwerpen te herhalen en te consolideren.

**Extra Recources/Materiaal:**
- Document 1
- V

In [22]:
# Print out a log of our conversation so far

conversation.pretty_print_conversation_history()

user:
Wat kun je voor mij doen?
assistant:
Als assistent kan ik je op verschillende manieren helpen:

1. Lesplan Maken: Ik kan je helpen bij het organiseren en maken van een gedetailleerd lesplan. Dit kan variëren van het plannen van een enkele les tot het ontwerpen van een volledig curriculum, inclusief het indelen van activiteiten, het vaststellen van leerdoelen en het aangeven van eventuele benodigde materialen.

2. Notuleren: Ik kan je helpen bij het bijhouden van vergaderingen of klassendiscussies. Dit kan inhouden dat ik belangrijke punten, genomen beslissingen en toegewezen taken noteer, of samenvattingen maak na afloop.

3. Beheren van agenda's: Ik kan helpen bij het plannen en bijhouden van afspraken, belangrijke data en deadlines, ervoor zorgen dat je georganiseerd blijft.

4. Onlinediensten: Ik kan je helpen bij het navigeren door het digitale landschap, zoals het zoeken naar bronnen voor lesmateriaal, het gebruik van educatieve apps en tools, en het begrijpen van het gebrui

### Knowledge retrieval

Now we'll extend the class to call a downstream service when a stop sequence is spoken by the Chatbot.

The main changes are:
- The system message is more comprehensive, giving criteria for the Chatbot to advance the conversation
- Adding an explicit stop sequence for it to use when it has the info it needs
- Extending the class with a function ```_get_search_results``` which sources Redis results

In [23]:
# Updated system prompt requiring Question and Year to be extracted from the user
system_prompt = '''
Je bent een vriendelijke en behulpzame instructiecoach die docenten helpt bij het plannen van een les.  Je hebt een boek tot je beschikking: 'Docentenhandleiding Burgerschap'. Je maakt een lesplan over een Thema, en in het bijzonder een lesplan behorend tot 1 van de 5 lessen die over een thema gaat. De doelgroep bestaat uit niveau 1 MBO studenten.  Ze hebben geen voorkennis van het onderwerp. Gebruik een helder leerdoel want dat is wat de studenten begrijpen of kunnen doen na de les. Maak nu met deze informatie en met de kennis uit de inhoud een aangepast lesplan in markdown formaat met een verscheidenheid aan lestechnieken en -modaliteiten, waaronder directe instructie, controleren op begrip (inclusief het verzamelen van bewijs van begrip van een brede steekproef van studenten), discussie, een boeiende activiteit in de klas en een opdracht.  Leg uit waarom je specifiek voor elk kiest.   Probeer het niet groter te maken dan 2  A4-tjes 

'''

# New Assistant class to add a vector database call to its responses
class RetrievalAssistant:
    
    def __init__(self):
        self.conversation_history = []  

    def _get_assistant_response(self, prompt):
        
        try:
            completion = openai.ChatCompletion.create(
              model=CHAT_MODEL,
              messages=prompt,
              temperature=0.1
            )
            
            response_message = Message(completion['choices'][0]['message']['role'],completion['choices'][0]['message']['content'])
            return response_message.message()
            
        except Exception as e:
            
            return f'Request failed with exception {e}'
    
    # The function to retrieve Redis search results
    def _get_search_results(self,prompt):
        latest_question = prompt
        search_content = get_redis_results(redis_client,latest_question,INDEX_NAME)['result'][0]
        return search_content
        

    def ask_assistant(self, next_user_prompt):
        [self.conversation_history.append(x) for x in next_user_prompt]
        assistant_response = self._get_assistant_response(self.conversation_history)
        
        # Answer normally unless the trigger sequence is used "searching_for_answers"
        if 'searching for answers' in assistant_response['content'].lower():
            question_extract = openai.Completion.create(model=COMPLETIONS_MODEL,prompt=f"Extraheer de laatste vraag van de user om een lesplan te maken, en gebruik dan deze conversation: {self.conversation_history}.")
            search_result = self._get_search_results(question_extract['choices'][0]['text'])
            
            # We insert an extra system prompt here to give fresh context to the Chatbot on how to use the Redis results
            # In this instance we add it to the conversation history, but in production it may be better to hide
            self.conversation_history.insert(-1,{"role": 'system',"content": f"Beantwoord de vraag door deze context te gebruiken: {search_result}. Als je het antwoord niet weet, zeg dan 'Sorry, Ik weet hierop niet het antwoord'"})
            #[self.conversation_history.append(x) for x in next_user_prompt]
            
            assistant_response = self._get_assistant_response(self.conversation_history)
            print(next_user_prompt)
            print(assistant_response)
            self.conversation_history.append(assistant_response)
            return assistant_response
        else:
            self.conversation_history.append(assistant_response)
            return assistant_response
            
        
    def pretty_print_conversation_history(self, colorize_assistant_replies=True):
        for entry in self.conversation_history:
            if entry['role'] == 'system':
                pass
            else:
                prefix = entry['role']
                content = entry['content']
                output = colored(prefix +':\n' + content, 'green') if colorize_assistant_replies and entry['role'] == 'assistant' else prefix +':\n' + content
                #prefix = entry['role']
                print(output)

In [24]:
conversation = RetrievalAssistant()
messages = []
system_message = Message('system',system_prompt)
user_message = Message('user',"Hoeveel thema's worden in Burgerschap behandeld? ")
messages.append(system_message.message())
messages.append(user_message.message())
response_message = conversation.ask_assistant(messages)
response_message

{'role': 'assistant',
 'content': "In de handleiding 'Docentenhandleiding Burgerschap' worden vijf thema's behandeld."}

In [27]:
messages = []
user_message = Message('user',"Kun je de vijf thema's puntsgewijs noemen?.")
messages.append(user_message.message())
response_message = conversation.ask_assistant(messages)
response_message

{'role': 'assistant',
 'content': "Natuurlijk! Hier zijn de vijf thema's uit de 'Docentenhandleiding Burgerschap':\n\n1. Identiteit en diversiteit\n2. Democratie en rechtsstaat\n3. Duurzaamheid en leefomgeving\n4. Gezondheid en welzijn\n5. Politiek en beleid"}

In [28]:
conversation.pretty_print_conversation_history()

user:
Hoeveel thema's worden in Burgerschap behandeld? 
assistant:
In de handleiding 'Docentenhandleiding Burgerschap' worden vijf thema's behandeld.
user:
Kun je de vijf thema's puntsgewijs noemen?.
assistant:
Natuurlijk! Hier zijn de vijf thema's uit de 'Docentenhandleiding Burgerschap':

1. Identiteit en diversiteit
2. Democratie en rechtsstaat
3. Duurzaamheid en leefomgeving
4. Gezondheid en welzijn
5. Politiek en beleid
user:
Kun je de vijf thema's puntsgewijs noemen?.
assistant:
Natuurlijk! Hier zijn de vijf thema's uit de 'Docentenhandleiding Burgerschap':

1. Identiteit en diversiteit
2. Democratie en rechtsstaat
3. Duurzaamheid en leefomgeving
4. Gezondheid en welzijn
5. Politiek en beleid


### Chatbot

Now we'll put all this into action with a real (basic) Chatbot.

In the directory containing this app, execute ```streamlit run chat.py```. This will open up a Streamlit app in your browser where you can ask questions of your embedded data. 

__Example Questions__:
- what is the cost cap for a power unit in 2023
- what should competitors include on their application form
- how can a competitor be disqualified

### Consolidation

Over the course of this notebook you have:
- Laid the foundations of your product by embedding our knowledge base
- Created a Q&A application to serve basic use cases
- Extended this to be an interactive Chatbot

These are the foundational building blocks of any Q&A or Chat application using our APIs - these are your starting point, and we look forward to seeing what you build with them!